In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-124461")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-124461
Azure region: southcentralus
Subscription id: 0c5a644d-c5ce-4e3b-bf42-4cb265317817
Resource group: aml-quickstarts-124461


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "udacity-projects"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(0.01, 1),
        '--max_iter': choice(100, 500, 1000, 5000),
        '--penalty': choice('l1', 'l2'),
        '--solver': choice('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")
    
# Create a SKLearn estimator for use with train.py


est = SKLearn(source_directory='.',
              entry_script='train.py',
              compute_target = aml_compute
             )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                                     hyperparameter_sampling=ps, 
                                     policy=policy, 
                                     primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=20,
                                     max_concurrent_runs=4)



In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.widgets import RunDetails

hyperDrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperDrive_run).show()
hyperDrive_run.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e49e4e90-22f6-4dc1-b162-a9f2f2d9c1e9
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_e49e4e90-22f6-4dc1-b162-a9f2f2d9c1e9?wsid=/subscriptions/0c5a644d-c5ce-4e3b-bf42-4cb265317817/resourcegroups/aml-quickstarts-124461/workspaces/quick-starts-ws-124461

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-04T00:49:38.040484][API][INFO]Experiment created<END>\n""<START>[2020-11-04T00:49:38.599739][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2020-11-04T00:49:39.1598041Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-11-04T00:49:38.884063][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_e49e4e90-22f6-4dc1-b162-a9f2f2d9c1e9
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_e49e4e90-22f6-4dc1-b162-a9f2f2d9c1e9?wsid=/subscriptions/0c5a6

{'runId': 'HD_e49e4e90-22f6-4dc1-b162-a9f2f2d9c1e9',
 'target': 'udacity-projects',
 'status': 'Completed',
 'startTimeUtc': '2020-11-04T00:49:37.747127Z',
 'endTimeUtc': '2020-11-04T01:06:26.62664Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1f8c4b44-9f42-48f3-bc85-d56cf52967a3',
  'score': '0.9122913505311078',
  'best_child_run_id': 'HD_e49e4e90-22f6-4dc1-b162-a9f2f2d9c1e9_16',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg124461.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_e49e4e90-22f6-4dc1-b162-a9f2f2

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_hyper_drive_model = hyperDrive_run.get_best_run_by_primary_metric()

print(best_hyper_drive_model.get_file_names())

filename = 'best_hyper_drive_model'
best_model = best_hyper_drive_model.register_model(model_name=filename, model_path="./")

['azureml-logs/55_azureml-execution-tvmps_350e803bf784c6a380faf38ba850faae1a1c01153657a6f7d5dfc9bb6fb4dcf8_d.txt', 'azureml-logs/65_job_prep-tvmps_350e803bf784c6a380faf38ba850faae1a1c01153657a6f7d5dfc9bb6fb4dcf8_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_350e803bf784c6a380faf38ba850faae1a1c01153657a6f7d5dfc9bb6fb4dcf8_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/107_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(data_path)

In [4]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=223)
train_data = pd.concat([x_train, y_train], axis=1)

In [6]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5)

In [8]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output = False)
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_ecbcb089-0e39-415c-b1f7-94297a810af9',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-11-04T01:26:29.178666Z',
 'endTimeUtc': '2020-11-04T01:59:30.564896Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"0c5a644d-c5ce-4e3b-bf42-4cb265317817","resource_group":"aml-quickstarts-124461","workspace_name":"quick-starts-ws-124461","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y

In [9]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_auto_ml_run, fitted_model = remote_run.get_output()

best_model_automl = best_auto_ml_run.register_model(model_name="auto_ml_best_run.sav", model_path="./")

In [10]:
aml_compute.delete()